## Live Demo

After the you run the cell below, a button will appear to initiate live face detection. When you click the button, a pop-up will appear showing the webcam feed with any detected faces framed by a green square.

To close the pop-up, press the "Q". Please note, the only way to close the pop-up is by pressing "Q".

This requires a working webcam.

In [ ]:
# To start live face detection
# Click on this cell and either press Shift + Enter 
# Or click the Run button in the toolbar above 

import cv2 as cv
import ipywidgets as widgets

CASCADE = cv.CascadeClassifier('haarcascade_frontalface_default.xml')


widget_output = widgets.Output()

live_button = widgets.Button(
    description='Live Detection',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Live face detection',
    icon='video-camera' # FontAwesome names without the `fa-` prefix
)

def start_live(change):
    webcam = cv.VideoCapture(0)
    
    while True:
        ret, frame = webcam.read()
        grayscale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        detected_faces = CASCADE.detectMultiScale(grayscale)

        for (x, y, w, h) in detected_faces:
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv.imshow('Live Detection', frame)
            
        # Press q to quit
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
                
    webcam.release()
    cv.destroyAllWindows()
    
live_button.on_click(start_live)
display(live_button)    

In [ ]:
#Examples 

img = 'test_images/test_faces3.jpeg'
pil_img = Image.open(img)

with open(img, 'rb') as img:
    img_bytestring = img.read()
np_array = np.frombuffer(img_bytestring, dtype=np.uint8)
grayscale = cv.imdecode(np_array, cv.IMREAD_GRAYSCALE)  # Convert 1-D np array to 2-D numpy array
detected_faces = CASCADE.detectMultiScale(grayscale)
    
img_copy = pil_img.copy() # Make copy of original image for drawing
draw_context = ImageDraw.Draw(img_copy) # Set up drawing context

# Use list of detected faces to draw boxes on img_copy, return
for face in detected_faces:
    draw_context.rectangle((face[0],face[1],face[0]+face[2],face[1]+face[3]), outline='red', width=2)
display(img_copy)  

In [ ]:
import io
import os

import cv2 as cv
import ipywidgets as widgets
import numpy as np
from PIL import Image, ImageDraw

CASCADE = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

# WIDGETS 
uploader = widgets.FileUpload(button_style='info')
scale_slider = widgets.FloatSlider(
    value=1.05, 
    min=1.05,  # Must be >=1.01
    max=1.50,
    step=0.05,
    description='Scale Factor',
    continuous_update=False,  # Debounce for slider handler
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)
neighbor_slider = widgets.IntSlider(
    value=3,  # Set default slider value to default minNeighbors argument
    min=1,
    max=10,
    step=1,
    description='Neighbors',
    continuous_update=False,  # Debounce for slider handler
    orientation='horizontal',
    readout=True
)
image_selector = widgets.Dropdown(
    options=[('Select a test image...', 0), ('Example 1', 'test_faces0.jpeg'), ('Example 2', 'test_faces1.jpeg' ), ('Example 3', 'test_faces3.jpeg'), ('Example 4', 'test_faces_cartoon.jpg')],
    value=0,
    description='Example Image:'
)
widget_output = widgets.Output()

# DETECTION FUNCTIONS 
def get_example_formats(selection):
    path = os.path.join('test_images', selection)
    pil_img = Image.open(path)

    with open(path, 'rb') as f:
        f = f.read()
        np_array = np.frombuffer(f, dtype=np.uint8)
        grayscale = cv.imdecode(np_array, cv.IMREAD_GRAYSCALE)  # Convert 1-D np array to 2-D numpy array
        
    formats = {"pil_img": pil_img, "grayscale": grayscale}
    return formats

def draw_example_faces(selection, scale_factor, min_neighbors):
    formats = get_example_formats(selection)
    detected_faces = CASCADE.detectMultiScale(formats['grayscale'],
                                              scale_factor, min_neighbors)
    
    img_copy = formats['pil_img'].copy() # Make copy of original image for drawing
    draw_context = ImageDraw.Draw(img_copy) # Set up drawing context

    # Use list of detected faces to draw boxes on img_copy, return
    for face in detected_faces:
        draw_context.rectangle((face[0],face[1],face[0]+face[2],face[1]+face[3]), outline='red', width=2)
    return img_copy 
    
def get_upload_formats():
    """Return PIL and grayscale formats for uploaded image."""
    img_bytestring = uploader.data[0]  # Get bytestring from uploader widget
    pil_img = Image.open(io.BytesIO(img_bytestring))  # Convert bytestring to PIL image
    np_array = np.frombuffer(img_bytestring, dtype=np.uint8)  # Convert bytestring to 1-D numpy array
    grayscale = cv.imdecode(np_array, cv.IMREAD_GRAYSCALE)  # Convert 1-D np array to 2-D numpy array
    formats = {"pil_img": pil_img, "grayscale": grayscale}
    return formats

def draw_faces(scale_factor, min_neighbors):
    """Detect faces and return copy of image with boxes drawn around faces."""
    formats = get_upload_formats()
    detected_faces = CASCADE.detectMultiScale(formats['grayscale'],
                                              scale_factor, min_neighbors)
    
    img_copy = formats['pil_img'].copy() # Make copy of original image for drawing
    draw_context = ImageDraw.Draw(img_copy) # Set up drawing context

    # Use list of detected faces to draw boxes on img_copy, return
    for face in detected_faces:
        draw_context.rectangle((face[0],face[1],face[0]+face[2],face[1]+face[3]), outline='red', width=2)
    return img_copy 

# HANDLER FUNCTIONS FOR WIDGETS
def handle_img_selection(change):
    scale_slider.value = 1.05
    neighbor_slider.value = 3
    
    with widget_output:
        display(draw_example_faces(change['new'], scale_slider.value, neighbor_slider.value))
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed

def handle_scale_slider(change):
    with widget_output:
        try: 
            display(draw_faces(change['new'], neighbor_slider.value))
        except IndexError: # Handle slider movement w/o upload
            display('Please upload an image first.')
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed
    
def handle_neighbor_slider(change):
    with widget_output:
        try:      
            display(draw_faces(scale_slider.value, change['new']))
        except IndexError: # Handle slider movement w/o upload
            display('Please upload an image first.')
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed
    
def handle_upload(change):
    # Reset values after >1 upload
    scale_slider.value = 1.05
    neighbor_slider.value = 3
    
    with widget_output:
        display(draw_faces(scale_slider.value, neighbor_slider.value))
    widget_output.clear_output(wait=True) # Clear previous output when new output is displayed
      
### OBSERVERS (LINKS WIDGETS TO HANDLER FUNCTIONS)
scale_slider.observe(handle_scale_slider, names='value')
neighbor_slider.observe(handle_neighbor_slider, names='value')
uploader.observe(handle_upload, names='_counter')
image_selector.observe(handle_img_selection, names='value')

### DISPLAY WIDGETS, OUTPUT
widgets.VBox([widgets.Label('Upload an image.'),
              uploader, image_selector, scale_slider, neighbor_slider, widget_output])